In [ ]:
import pandas as pd

# Load the uploaded CSV file
file_path = "data/ELSI_funding_and_more.csv"
df = pd.read_csv(file_path)

# Show the first few rows of the dataframe
df.head()

,Agency Name,State Name [District] Latest available year,State Name [District] 2023-24,County Name [District] 2023-24,Agency ID - NCES Assigned [District] Latest available year,Agency Name [District] 2023-24,Total Students All Grades (Excludes AE) [District] 2023-24,Total Students All Grades (Excludes AE) [District] 2022-23,Total Students All Grades (Excludes AE) [District] 2021-22,Total Students All Grades (Excludes AE) [District] 2020-21,...,State Rev. - General Formula Assistance (C01) [District Finance] 2015-16,State Rev. - General Formula Assistance (C01) [District Finance] 2014-15,Local Rev. - Property Taxes (T06) [District Finance] 2021-22,Local Rev. - Property Taxes (T06) [District Finance] 2020-21,Local Rev. - Property Taxes (T06) [District Finance] 2019-20,Local Rev. - Property Taxes (T06) [District Finance] 2018-19,Local Rev. - Property Taxes (T06) [District Finance] 2017-18,Local Rev. - Property Taxes (T06) [District Finance] 2016-17,Local Rev. - Property Taxes (T06) [District Finance] 2015-16,Local Rev. - Property Taxes (T06) [District Finance] 2014-15
0,(ILC) ATOKA-COAL COUNTIES,Oklahoma,†,†,4000760,†,†,†,†,†,...,†,0,†,†,†,†,†,†,†,†
1,21ST CENTURY CHARTER SCH OF GARY,Indiana,INDIANA,Marion County,1800046,21st Century Charter Sch of Gary,1213,1337,1329,1264,...,5585000,5369000,†,†,†,†,†,†,†,†
2,21ST CENTURY CYBER CS,Pennsylvania,PENNSYLVANIA,Chester County,4200091,21st Century Cyber CS,1103,1239,1536,2503,...,0,0,†,†,†,†,†,†,†,†
3,21ST CENTURY PUBLIC ACADEMY,NEW MEXICO,NEW MEXICO,Bernalillo County,3500187,21ST CENTURY PUBLIC ACADEMY,378,393,364,351,...,†,†,†,†,†,†,†,†,†,†
4,A CENTER FOR CREATIVE EDUCATION (79457),Arizona,†,†,400328,†,†,†,†,†,...,509000,404000,†,†,†,†,†,†,†,†


In [22]:
# Define target states
target_states = {"ARIZONA", "FLORIDA", "HAWAII", "INDIANA"}
# filter by state (some are capitalized, some not)
filtered_df = df[df["State Name [District] Latest available year"].str.upper(
).isin(target_states)]

# Drop the redundant and potentially inconsistent 2023-24 state name column
if "State Name [District] 2023-24" in filtered_df.columns:
    filtered_df = filtered_df.drop(columns=["State Name [District] 2023-24"])

# Check the number of remaining rows and show a sample
filtered_df.shape

(645, 269)

In [ ]:
import pandas as pd
import re

# Identify time-series columns (those with years like 2023-24, etc.)
time_series_cols = [col for col in filtered_df.columns if re.search(
    r'\b20\d{2}-\d{2}\b', col)]

# ID columns to retain
id_columns = [
    "Agency Name",
    "Agency ID - NCES Assigned [District] Latest available year",
    "State Name [District] Latest available year"
]

# Melt to long format
df_long = filtered_df.melt(
    id_vars=id_columns,
    value_vars=time_series_cols,
    var_name="Raw Column Name",
    value_name="Value"
)

# Extract year and metric name
df_long["Year"] = df_long["Raw Column Name"].str.extract(r'(20\d{2}-\d{2})')
df_long["Metric"] = df_long["Raw Column Name"].str.replace(
    r'\s*\[.*?\]\s*20\d{2}-\d{2}', '', regex=True).str.strip()

# Drop rows without valid year
df_long = df_long.dropna(subset=["Year"])

# Pivot so each row is one district-year, and each metric is a column
pivoted = df_long.pivot_table(
    index=["Agency Name",
           "Agency ID - NCES Assigned [District] Latest available year",
           "State Name [District] Latest available year",
           "Year"],
    columns="Metric",
    values="Value",
    aggfunc="first"
)

# Drop duplicates in columns before reset_index
for col in pivoted.index.names:
    if col in pivoted.columns:
        del pivoted[col]

# Reset index safely
reshaped_df = pivoted.reset_index()
reshaped_df.columns.name = None  # remove MultiIndex label

In [24]:
reshaped_df.columns

Index(['Agency Name',
       'Agency ID - NCES Assigned [District] Latest available year',
       'State Name [District] Latest available year', 'Year',
       'American Indian/Alaska Native - female',
       'American Indian/Alaska Native - male',
       'Asian or Asian/Pacific Islander - female',
       'Asian or Asian/Pacific Islander - male',
       'Black or African American - female',
       'Black or African American - male', 'County Name',
       'Federal Rev. - Federal Title I Revenue (C14)', 'Hispanic - female',
       'Hispanic - male', 'Local Rev. - Property Taxes (T06)',
       'Nat. Hawaiian or Other Pacific Isl. - female',
       'Nat. Hawaiian or Other Pacific Isl. - male', 'Pupil/Teacher Ratio',
       'State Rev. - General Formula Assistance (C01)',
       'Total Dropouts  Grades 9-12',
       'Total Revenue (TOTALREV) per Pupil (V33)',
       'Total Students All Grades (Excludes AE)', 'Two or More Races - female',
       'Two or More Races - male', 'White - female', 

In [21]:
reshaped_df

,Agency Name,Agency ID - NCES Assigned [District] Latest available year,State Name [District] Latest available year,Year,American Indian/Alaska Native - female,American Indian/Alaska Native - male,Asian or Asian/Pacific Islander - female,Asian or Asian/Pacific Islander - male,Black or African American - female,Black or African American - male,...,Pupil/Teacher Ratio,State Name,State Rev. - General Formula Assistance (C01),Total Dropouts Grades 9-12,Total Revenue (TOTALREV) per Pupil (V33),Total Students All Grades (Excludes AE),Two or More Races - female,Two or More Races - male,White - female,White - male
0,21ST CENTURY CHARTER SCH OF GARY,1800046,Indiana,2014-15,0,1,0,0,329,358,...,†,NaN,5369000,NaN,12591,709,5,5,0,0
1,21ST CENTURY CHARTER SCH OF GARY,1800046,Indiana,2015-16,0,1,0,0,357,377,...,379,NaN,5585000,NaN,10724,758,3,5,0,0
2,21ST CENTURY CHARTER SCH OF GARY,1800046,Indiana,2016-17,–,1,–,–,430,430,...,30.62,NaN,7045000,NaN,10520,888,6,7,1,2
3,21ST CENTURY CHARTER SCH OF GARY,1800046,Indiana,2017-18,–,1,–,–,422,462,...,28.53,NaN,7424000,NaN,10833,934,18,9,2,1
4,21ST CENTURY CHARTER SCH OF GARY,1800046,Indiana,2018-19,–,–,–,–,408,429,...,26.73,NaN,7585000,‡,11713,883,14,9,–,–
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6445,INTELLI-SCHOOL INC. (4352),400061,Arizona,2019-20,2,1,0,0,0,2,...,†,NaN,963000,NaN,10250,104,1,4,23,38
6446,INTELLI-SCHOOL INC. (4352),400061,Arizona,2020-21,3,2,0,0,4,5,...,–,NaN,976000,NaN,11300,100,0,6,19,29
6447,INTELLI-SCHOOL INC. (4352),400061,Arizona,2021-22,0,0,0,0,0,0,...,–,NaN,769000,NaN,12702,84,0,4,26,16
6448,INTELLI-SCHOOL INC. (4352),400061,Arizona,2022-23,0,0,0,0,0,0,...,–,NaN,NaN,22,NaN,67,1,2,20,13
